In [1]:
!pip3 install opencv-python

In [11]:
import cv2
import numpy as np
import os
import glob
 
# Defining the dimensions of checkerboard
CHECKERBOARD = (6,9)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 

# defining world points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
 


In [12]:
# Extracting path of individual image stored in a given directory
images = glob.glob('./Calibration_pictures/*.JPG')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
#     cv2.imshow('img',img)
#     cv2.waitKey(1)
#     cv2.destroyAllWindows()

# cv2.destroyAllWindows()
# cv2.waitKey(1)
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[3.08392661e+03 0.00000000e+00 1.98378736e+03]
 [0.00000000e+00 3.07897380e+03 1.49240126e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[ 1.38991085e-01 -6.27088713e-01 -8.20952916e-04  1.78121606e-04
   7.51806830e-01]]
rvecs : 

(array([[0.32430171],
       [0.20347183],
       [1.55843014]]), array([[ 0.03250604],
       [-0.04917548],
       [ 1.58310811]]), array([[-0.37024367],
       [-0.54411988],
       [ 1.49340813]]), array([[ 0.24122406],
       [-0.28282813],
       [ 1.55885191]]), array([[-0.1203122 ],
       [-0.32858672],
       [ 1.53363162]]), array([[0.50862604],
       [0.35718196],
       [1.53422464]]), array([[ 0.68930594],
       [-0.18386573],
       [ 1.42248062]]), array([[-0.47518011],
       [-0.2807595 ],
       [ 1.57839137]]), array([[ 0.36120091],
       [-0.38373537],
       [ 1.53571713]]), array([[0.25784291],
       [0.47751564],
       [1.42478448]]), array([[-0.17998585],
       [ 0.24325408],
       [ 1.57289

In [4]:
print(mtx)

[[3.08392661e+03 0.00000000e+00 1.98378736e+03]
 [0.00000000e+00 3.07897380e+03 1.49240126e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [13]:
print(f"fx: {mtx[0][0]}, fy: {mtx[1][1]}")
print(f"cx: {mtx[0][2]}, cy: {mtx[1][2]}")

fx = mtx[0][0]
fy = mtx[1][1]
cx = mtx[0][2]
cy = mtx[1][2]

fx: 3083.926607105556, fy: 3078.973797704206
cx: 1983.7873623042801, cy: 1492.4012603321164


In [16]:
from ultralytics import YOLO
import os

model = YOLO('yolov8x.pt')


ModuleNotFoundError: No module named 'ultralytics'

In [9]:
def convert_to_real_world_coordinates(x, y, d_pix, fx, fy, cx, cy, true_diameter):
    # Calculate real-world coordinates
    Z = (fx * true_diameter) / d_pix
    X = ((x - cx) * Z) / fx
    Y = ((y - cy) * Z) / fy

    return X, Y, Z

def meters_to_inches(meters):
    return meters * 39.3701  

def process_image(image_path):
    # Load the image with OpenCV
    current_frame = cv2.imread(image_path)
    
    # Ensure the image was loaded
    if current_frame is None:
        print(f"Failed to load image {image_path}")
        return
    
    # Predict using the model for baseball class (class_id 32)
    results = model.predict(current_frame, classes=32)
    
    # Iterate through the results
    for result in results:
        boxes = result.boxes
        if boxes.conf.size(0) > 0:
            # There are detections
            for i in range(boxes.xyxy.size(0)): # For each detection
                # Extract bounding box coordinates
                x1, y1, x2, y2 = map(int, boxes.xyxy[i].tolist())
                
                # Calculate the diameter of the baseball (approximation)
                d_pix = ((x2 - x1) + (y2 - y1)) / 2
                
                # Draw rectangle around the baseball
                cv2.rectangle(current_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                print(f"Diameter of baseball in pixels: {d_pix}")

                # get midpoint of the ball in the image (pixels)
                y = y1 + (y2 - y1) / 2
                x = x1 + (x2 - x1) / 2
                
                # calculate real-world depth
                X, Y, Z = convert_to_real_world_coordinates(x, y, 0, d_pix, fx, fy, cx, cy)

                print(f"Real-world coordinates. X:{X} Y:{Y} Z:{Z}")
                cv2.putText(current_frame, f"Real world coordinates:", (x1, y1-400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
                cv2.putText(current_frame, f"X:{X}", (x1, y1-300), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
                cv2.putText(current_frame, f"Y:{Y}", (x1, y1-200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
                cv2.putText(current_frame, f"Z:{Z}", (x1, y1-100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

                
    # Display the modified frame with bounding boxes
    cv2.imshow("Detected Baseball", current_frame)
    cv2.waitKey(0)  # Wait for a key press to close
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [6]:
# Directory containing the images
directory_path = 'test_images'

# Check if the directory exists
if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist.")
else:
    # Process each image in the directory
    for filename in os.listdir(directory_path):
        if filename.lower().endswith((".jpg", ".png")):
            image_path = os.path.join(directory_path, filename)
            process_image(image_path)


NameError: name 'model' is not defined

In [7]:
!pip3 install inference --break-system-packages

In [4]:
# from inference_sdk import InferenceHTTPClient
from inference.models.utils import get_roboflow_model
import cv2


In [5]:
# Roboflow model
model_name = "pingpong-t6uto/1"
# model_version = "18"

# Get Roboflow face model (this will fetch the model from Roboflow)
model = get_roboflow_model(
    model_id="{}".format(model_name),
    api_key="JIcwLHeghA4eR7h4ye7S"
)

In [6]:
# Image path
image_path = "test_images/IMG_5201.JPG"

# Load image with opencv
frame = cv2.imread(image_path)

# Inference image to find faces
results = model.infer(image=frame,
                        confidence=0.5,
                        iou_threshold=0.5)

if results:
  for detection in results[0].predictions:
    x0 = int(detection.x - (detection.width / 2))
    x1 = int(detection.x + (detection.width / 2))
    y0 = int(detection.y - (detection.height / 2))
    y1 = int(detection.y + (detection.height / 2))
    print(f"{x0} {x1} {y0} {y1}")

    cv2.rectangle(frame, (x0, y0), (x1, y1), (255,255,0), 10)
    cv2.putText(frame, "Ball", (x0, y0 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

# Show image
cv2.imshow('Image Frame', frame)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1)

984 1121 2484 2633
2402 2526 2255 2392
2274 2425 2747 2914
1286 1395 2077 2201
1973 2068 1803 1909


-1

In [7]:
import numpy as np
from scipy.spatial.distance import pdist, squareform


In [17]:
points = []

cv2.imshow('Image Frame', frame)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1)

if results:
  for detection in results[0].predictions:
    # calculate real-world depth
    X, Y, Z = convert_to_real_world_coordinates(detection.x, detection.y, detection.width, fx, fy, cx, cy, 0.04)
    X, Y, Z = meters_to_inches(X), meters_to_inches(Y), meters_to_inches(Z)
    points.append([X, Y, Z])
    print(f"Real-world coordinates. X:{X} Y:{Y} Z:{Z}")

    # display real-world coordinates
    cv2.putText(frame, f"Real world coordinates for point {len(points) - 1}", (int(detection.x), int(detection.y - 400)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    cv2.putText(frame, f"X:{X}", (int(detection.x), int(detection.y - 300)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    cv2.putText(frame, f"Y:{Y}", (int(detection.x), int(detection.y - 200)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    cv2.putText(frame, f"Z:{Z}", (int(detection.x), int(detection.y - 100)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

# Show image
print("here")
cv2.imshow('Image Frame', frame)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1)

Real-world coordinates. X:-10.705073454789998 Y:12.274432244116495 Z:35.44948873413327
Real-world coordinates. X:6.098715989465891 Y:10.57195945006837 Z:39.16596739174401
Real-world coordinates. X:3.8140776469786117 Y:13.977701589747259 Z:32.16278116937919
Real-world coordinates. X:-9.294050562442473 Y:9.356919775726954 Z:44.555779418130804
Real-world coordinates. X:0.6085811441449521 Y:6.03702972185463 Z:51.12189427975009
here


-1

In [16]:
distance_matrix = squareform(pdist(points, 'euclidean'))

expected_distances = np.array([12, 12, 18.98, 18.98])

def compute_mse_for_point(point_index, distance_matrix, expected_distances):
    # check for only 5 points
    if len(distance_matrix) != 5:
        raise ValueError("Distance matrix must have 5 points corresponding to the corners of the home plate.")

    actual_distances = np.sort(np.delete(distance_matrix[point_index], point_index))
    # print(actual_distances)

    mse = np.mean((actual_distances - expected_distances) ** 2)
    return mse

# Compute MSE for each point being the front tip
mse_scores = []
for i in range(len(points)):
    mse = compute_mse_for_point(i, distance_matrix, expected_distances)
    mse_scores.append(mse)
    print(f"MSE for point {i} as back tip: {mse}")

# Determine which point has the lowest MSE
back_tip_index = np.argmin(mse_scores)
print(f"Point {back_tip_index} is likely the front tip of the home plate based on MSE.")

# distances = np.sort(np.delete(distance_matrix[back_tip_index], back_tip_index))
# close_back_left_tip_index = np.where(distance_matrix[back_tip_index] == distances[0])[0][0]
# print(f"Closest point to back left tip: {close_back_left_tip_index}")


# Identified front tip index from previous steps
front_tip_index = back_tip_index  # Replace back_tip_index with the variable you use

# Home Plate Configuration:
#           3           2
#           _____________
#           |           | 
#    (left) |           | (right)
#           |           |
#           4           1
#            \         /
#             \       /
#              \     /
#               \   /
#                \ /
#                 0 (Front of the plate)
def determine_plate_corners(points, distance_matrix, front_tip_index):
    # check for only 5 points
    if len(distance_matrix) != 5:
        raise ValueError("Distance matrix must have 5 points corresponding to the corners of the home plate.")

    num_points = len(points)
    # Exclude front tip from possible back corners
    other_indices = [i for i in range(num_points) if i != front_tip_index]
    print(other_indices)

    # Calculate distances from front tip to other points
    distances_from_front = distance_matrix[front_tip_index, other_indices]
    print(distances_from_front)

    # Identify back corners as the two farthest points from the front tip
    back_corners_indices = np.argsort(-distances_from_front)[:2]  # Get indices of two largest distances
    back_corners = [other_indices[i] for i in back_corners_indices]

    # The remaining point is the opposite side corner
    remaining_index = list(set(other_indices) - set(back_corners))

    # find the closest point
    if np.linalg.norm(points[remaining_index[0]]) < np.linalg.norm(points[remaining_index[1]]):
        closest_point_index = remaining_index[0]
        further_point_index = remaining_index[1]
    else:
        closest_point_index = remaining_index[1]
        further_point_index = remaining_index[0]
    print(closest_point_index)

    # find the closest side
    if (distance_matrix[closest_point_index, back_corners[0]] < distance_matrix[closest_point_index, back_corners[1]]):
        closest_side_back_point_index = back_corners[0]
        further_side_back_point_index = back_corners[1]
    else:
        closest_side_back_point_index = back_corners[1]
        further_side_back_point_index = back_corners[0]

    # determine handed-ness of the batter
    if points[closest_point_index][0] < points[further_point_index][0]:  # compare x values of the closest point and the further point, if the x value of the further point is greater, we are standing on the left side of the plate and the batter is right-handed
        print("Right-handed batter (batter is on the left side of plate)")
        return [front_tip_index, further_point_index, further_side_back_point_index, closest_side_back_point_index, closest_point_index]
    else:
        print("Left-handed batter (batter is on the right side of plate)")
        return [front_tip_index, closest_point_index, closest_side_back_point_index, further_side_back_point_index, further_point_index]

# Determine corners
corners = determine_plate_corners(points, distance_matrix, front_tip_index)
print("Identified corners:", corners)

MSE for point 0 as back tip: 4.7411695474139774
MSE for point 1 as back tip: 7.124052726568821
MSE for point 2 as back tip: 6.860859801225601
MSE for point 3 as back tip: 3.146186353901201
MSE for point 4 as back tip: 2.219863034517643
Point 4 is likely the front tip of the home plate based on MSE.
[0, 1, 2, 3]
[20.31079187 13.91586706 20.80330376 12.33684113]
1
Left-handed batter (batter is on the right side of plate)
Identified corners: [4, 1, 2, 0, 3]
